# Introduction to the OmegaFormat

This notebook will introduce you to the OmegaFormat and the usage of the acopaning python library. The OmegaFormat is used to store and exchange vehicle traffic data. This includes data on objects with their trajectories in an object-list based disription and map information. The represention of geopositions is in an aboslute manner, relative to a reference point. 

To install run:`pip install -e ..[visualization]`

## Create a OmegaFormat file from scratch
The OmegaFormat uses the hdf5 file format. The first step to create a new data file is to define the `MetaData`.

In [5]:
from datetime import datetime
import omega_format

meta_data = omega_format.MetaData(recorder_number="1", 
                                recording_number="1",
                                daytime=datetime.now(),
                                reference_point_lat=50.786687,
                                reference_point_lon=6.046312)

The `meta_data` object stores information that helps trace the origin and collection condition of the data and it defines the reference point of the used coordinate system in geographic coordiante system. With the `meta_data` object we can create the ReferenceRecording, which is the container for all the data. Besidse the `meta_data` we need to set the timestamps as an array of time relative the daytime given in the meta data. the timestamps are referenced from all objects.

In [6]:
rr = omega_format.ReferenceRecording(meta_data=meta_data,
                                    timestamps=[0,1])

Now we can add a first road user to the reference recoding. As you can see, we only specify a birth index (index at which position in the timestamp array the road user begins existing) and no timestamps of the oberervations directly. This ensures that all objects share the same observation times.

In [7]:
rr.road_users['RU0'] = omega_format.RoadUser(type=omega_format.RoadUserType.CAR,
                                        sub_type=omega_format.RoadUserSubTypeGeneral.REGULAR,
                                        birth=0,
                                        bb=omega_format.BoundingBox.create(length=2, width=3, height=1),
                                        tr=omega_format.Trajectory(pos_x=[0,1],
                                                                    pos_y=[0,1],
                                                                    pos_z=[0,0],
                                                                    heading=[0,0])
)

We can add as many Road Users as we like. 

For the analysis of traffic, map information is crucial. The OmegaFormat also contains map information. Similar to the ASAM OpenDRIVE specification, we describe the map through roads which are themself made up of lanes. So we first create a Road:

In [8]:
rr.roads[0] = omega_format.Road(location=omega_format.RoadLocation.URBAN)

You now would go on and add Lanes to the Road. The geometry of each lane is defined through Boundaries (LineStrings/Polylines) like the Lanelet2 format. To explore more about how maps are defined we suggest to look into the example converters from OpenDRIVE in the `03_Converters.ipynb` notebook.

When we have added all the data, we can save the Information to an hdf5 file:

In [9]:
rr.to_hdf5('test.hdf5')

and read in read the created file back in with

In [10]:
rr = omega_format.ReferenceRecording.from_hdf5('test.hdf5')

## Visualization 

You can visulize ReferenceRecordings with the following code. A QT GUI Application will start where you can explore the recording.

In [ ]:
import omega_format
from omega_format.visualization import Visualizer, SnippetContainer
reference_recording = omega_format.ReferenceRecording.from_hdf5('test.hdf5')
visualizer = Visualizer(SnippetContainer.create_list(references=reference_recording))
visualizer.start_gui_and_visualization()